# Problem Statement
Whenever the Imaging core needs to do a big data transfer/delivery to outside of the imaging core, there needs to be a document (called a Manifest) outlining what is in the package of data. For example, the data package would have 2 studies for patient A, but the manifest states that there should be 3. In such cases, we would be able to catch discrepancies and ensure data integrity.

# Solution
Produce a manifest of studies with:

- patient name
- study uid
- study uuid
- modality
- study description
- image count
- study date
- study time

# Code 
### 1. Setup: get project's database

In [ ]:
import AMBRA_Backups
import pandas as pd
import re
from datetime import datetime

In [ ]:
db_name = "MOST"
db = AMBRA_Backups.database.Database(db_name)

In [ ]:
# Define columns in Manifest
columns = [
    "studies.id",
    "patient_name",
    "study_uid",
    "uuid",
    "modality",
    "study_description",
    "series_count",
    "study_date",
]

### 3. Get Manifest data from Database

In [ ]:
manifest_data = db.run_select_query(
    f"""
   SELECT {', '.join(columns)}
   FROM MOST.studies
   INNER JOIN patients on patients.id = studies.id_patient
   WHERE deleted = 0
   """
)
manifest_df = pd.DataFrame(columns=columns, data=manifest_data)

rename_columns = []

# Rename columns for Capitalization
for col in columns:
    split = re.split(r'[.|_]', col)
    split = [s.capitalize() for s in split]
    rename_columns.append(' '.join(split))

manifest_df.columns = rename_columns
manifest_df.to_excel(f'Manifest_{db_name}_{datetime.now()}.xlsx')